In [ ]:
!python3 -m pip install open_spiel


     |████████████████████████████████| 5.5 MB 12.4 MB/s            
     |████████████████████████████████| 77 kB 15.4 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94563 sha256=4019fadf5fd4760fcff924bb221afbf8ea509c973d0972e04fa595b3d06345fe
  Stored in directory: /Users/mihokoda/Library/Caches/pip/wheels/fd/c2/0d/5d94d95e5875ea17b85a9f1f99b8dd2e50517137c8042c6468
Successfully built ml-collections
You should consider upgrading via the '/Users/mihokoda/opt/miniconda3/bin/python3 -m pip install --upgrade pip' command.


In [5]:
import pyspiel

# List available games
print("OpenSpiel installed successfully!")
print(f"Number of games available: {len(pyspiel.registered_games())}")

# Test loading Kuhn Poker
game = pyspiel.load_game("kuhn_poker")
print(f"\nLoaded game: {game.get_type().short_name}")
print(f"Players: {game.num_players()}")

OpenSpiel installed successfully!
Number of games available: 113

Loaded game: kuhn_poker
Players: 2


In [8]:
# Load RPS game
game = pyspiel.load_game("matrix_rps")
state = game.new_initial_state()

print("Initial state:", state)
print("Current player:", state.current_player())  # Returns -2 for simultaneous

# For simultaneous games, apply actions together
state.apply_actions([0, 2])  # Player 0: Rock (0), Player 1: Scissors (2)

print("Game over:", state.is_terminal())
print("Returns:", state.returns())  # [1.0, -1.0] means P0 wins

Initial state: Terminal? false
Row actions: Rock Paper Scissors 
Col actions: Rock Paper Scissors 
Utility matrix:
0,0 -1,1 1,-1 
1,-1 0,0 -1,1 
-1,1 1,-1 0,0 

Current player: -2
Game over: True
Returns: [1.0, -1.0]


In [9]:
# Load Kuhn Poker
game = pyspiel.load_game("kuhn_poker")
state = game.new_initial_state()

print(f"Game: {game.get_type().long_name}")
print(f"Players: {game.num_players()}")
print(f"Initial state: {state}")

# See information about the game
print("\nGame info:")
print(f"- Min utility: {game.min_utility()}")
print(f"- Max utility: {game.max_utility()}")
print(f"- Max game length: {game.max_game_length()}")

Game: Kuhn Poker
Players: 2
Initial state: 

Game info:
- Min utility: -2.0
- Max utility: 2.0
- Max game length: 3


In [5]:
import pyspiel
import numpy as np
from collections import defaultdict

class CFRSolver:
    """
    Counterfactual Regret Minimization solver for 2-player zero-sum games.
    """
    def __init__(self, game):
        self.game = game
        self.num_players = game.num_players()
        
        # Regret and strategy tables indexed by information set string
        self.regret_sum = defaultdict(lambda: np.zeros(self.game.num_distinct_actions()))
        self.strategy_sum = defaultdict(lambda: np.zeros(self.game.num_distinct_actions()))
        self.info_set_map = {}  # Maps info state string to action list
        
        print(f"=== CFR Solver Initialized ===")
        print(f"Game: {game.get_type().long_name}")
        print(f"Players: {self.num_players}")
        print(f"Max actions: {game.num_distinct_actions()}")
        print()
    
    def train(self, num_iterations):
        """
        Train CFR for a specified number of iterations.
        """
        print(f"=== Starting CFR Training for {num_iterations} iterations ===\n")
        
        util = np.zeros(self.num_players)
        
        for iteration in range(num_iterations):
            # Run CFR from the root for each player
            for player in range(self.num_players):
                state = self.game.new_initial_state()
                util[player] = self.cfr(state, player, iteration)
            
            # Print progress every 10% of iterations
            if (iteration + 1) % max(1, num_iterations // 10) == 0:
                avg_game_value = np.mean(util)
                print(f"Iteration {iteration + 1}/{num_iterations} - Avg game value: {avg_game_value:.6f}")
        
        print(f"\n=== Training Complete ===")
        print(f"Final average game values: {util}")
        print()
    
    def cfr(self, state, player, iteration):
        """
        Counterfactual Regret Minimization recursive algorithm.
        Now handles both sequential and simultaneous games.
        """
        # Terminal state - return utility
        if state.is_terminal():
            return state.returns()[player]
        
        # Chance node - sample and recurse
        if state.is_chance_node():
            outcomes = state.chance_outcomes()
            action = np.random.choice([o[0] for o in outcomes], 
                                    p=[o[1] for o in outcomes])
            state.apply_action(action)
            return self.cfr(state, player, iteration)
        
        # HANDLE SIMULTANEOUS MOVES
        if state.is_simultaneous_node():
            return self.cfr_simultaneous(state, player, iteration)
        
        # Get current player (sequential game)
        current_player = state.current_player()
        
        # Get information set (what the player knows)
        info_state = state.information_state_string(current_player)
        legal_actions = state.legal_actions()
        num_actions = len(legal_actions)
        
        # Store legal actions for this info set
        if info_state not in self.info_set_map:
            self.info_set_map[info_state] = legal_actions
        
        # Get current strategy for this information set
        strategy = self.get_strategy(info_state, legal_actions)
        
        # Compute counterfactual values for each action
        action_values = np.zeros(num_actions)
        
        for i, action in enumerate(legal_actions):
            new_state = state.child(action)
            action_values[i] = self.cfr(new_state, player, iteration)
        
        # Compute expected value
        expected_value = np.dot(strategy, action_values)
        
        # Update regrets only for the player we're computing CFR for
        if current_player == player:
            # Regret = value(action) - value(current strategy)
            regrets = action_values - expected_value
            
            # Update cumulative regrets
            for i, action in enumerate(legal_actions):
                self.regret_sum[info_state][action] += regrets[i]
        
        # Update strategy sum for average strategy (for all players)
        for i, action in enumerate(legal_actions):
            self.strategy_sum[info_state][action] += strategy[i]
        
        return expected_value

    def cfr_simultaneous(self, state, player, iteration):
        """
        Handle simultaneous move nodes (like Rock-Paper-Scissors).
        """
        # Get legal actions for all players
        legal_actions_list = [state.legal_actions(p) for p in range(self.num_players)]
        
        # Get information sets and strategies for all players
        strategies = []
        info_states = []
        
        for p in range(self.num_players):
            info_state = state.information_state_string(p)
            info_states.append(info_state)
            legal_actions = legal_actions_list[p]
            
            if info_state not in self.info_set_map:
                self.info_set_map[info_state] = legal_actions
            
            strategy = self.get_strategy(info_state, legal_actions)
            strategies.append(strategy)
        
        # Compute counterfactual values for the target player
        target_legal_actions = legal_actions_list[player]
        num_actions = len(target_legal_actions)
        action_values = np.zeros(num_actions)
        
        # For each action of the target player
        for i, action in enumerate(target_legal_actions):
            value = 0.0
            
            # Sample opponent actions according to their strategies
            opponent_actions = []
            opponent_probs = 1.0
            
            for p in range(self.num_players):
                if p == player:
                    opponent_actions.append(action)
                else:
                    # Sample from opponent's strategy
                    opp_legal_actions = legal_actions_list[p]
                    opp_strategy = strategies[p]
                    opp_action_idx = np.random.choice(len(opp_legal_actions), p=opp_strategy)
                    opp_action = opp_legal_actions[opp_action_idx]
                    opponent_actions.append(opp_action)
                    opponent_probs *= opp_strategy[opp_action_idx]
            
            # Apply joint action
            new_state = state.clone()
            new_state.apply_actions(opponent_actions)
            
            # Recurse
            action_values[i] = self.cfr(new_state, player, iteration)
        
        # Get expected value under current strategy
        target_strategy = strategies[player]
        expected_value = np.dot(target_strategy, action_values)
        
        # Update regrets for target player
        info_state = info_states[player]
        regrets = action_values - expected_value
        
        for i, action in enumerate(target_legal_actions):
            self.regret_sum[info_state][action] += regrets[i]
        
        # Update strategy sum
        for i, action in enumerate(target_legal_actions):
            self.strategy_sum[info_state][action] += target_strategy[i]
        
        return expected_value
    
    def get_strategy(self, info_state, legal_actions):
        """
        Get current strategy using Regret Matching.
        Strategy is proportional to positive regrets.
        """
        num_actions = len(legal_actions)
        strategy = np.zeros(num_actions)
        
        # Get positive regrets
        regrets = self.regret_sum[info_state]
        positive_regrets = np.maximum(regrets[legal_actions], 0)
        
        sum_positive_regret = np.sum(positive_regrets)
        
        if sum_positive_regret > 0:
            # Strategy proportional to positive regrets
            strategy = positive_regrets / sum_positive_regret
        else:
            # Uniform random if no positive regrets
            strategy = np.ones(num_actions) / num_actions
        
        return strategy
    
    def get_average_strategy(self, info_state=None):
        """
        Get the average strategy (this converges to Nash equilibrium).
        If info_state is None, return all average strategies.
        """
        if info_state is not None:
            # Return strategy for specific info state
            legal_actions = self.info_set_map[info_state]
            strategy_sum = self.strategy_sum[info_state][legal_actions]
            
            sum_strategy = np.sum(strategy_sum)
            if sum_strategy > 0:
                return strategy_sum / sum_strategy
            else:
                return np.ones(len(legal_actions)) / len(legal_actions)
        else:
            # Return all strategies
            avg_strategies = {}
            for info_state in self.info_set_map.keys():
                avg_strategies[info_state] = self.get_average_strategy(info_state)
            return avg_strategies
    
    def print_strategy(self, player=None):
        """
        Print the learned average strategy in a readable format.
        """
        print(f"\n=== Average Strategy (Nash Equilibrium Approximation) ===")
        
        action_names = {
            0: "Pass/Check",
            1: "Bet/Call"
        }
        
        for info_state in sorted(self.info_set_map.keys()):
            # Filter by player if specified
            if player is not None and not info_state.startswith(str(player)):
                continue
                
            legal_actions = self.info_set_map[info_state]
            strategy = self.get_average_strategy(info_state)
            
            print(f"\nInfo Set: {info_state}")
            for i, action in enumerate(legal_actions):
                action_name = action_names.get(action, f"Action {action}")
                print(f"  {action_name}: {strategy[i]:.4f} ({strategy[i]*100:.2f}%)")


def compute_exploitability(cfr_solver, num_samples=1000):
    """
    Compute exploitability: how much can a best-response opponent exploit our strategy?
    Lower is better. 0 means Nash equilibrium.
    """
    print(f"\n=== Computing Exploitability ({num_samples} samples) ===")
    
    game = cfr_solver.game
    total_utility = np.zeros(cfr_solver.num_players)
    
    for _ in range(num_samples):
        state = game.new_initial_state()
        
        while not state.is_terminal():
            if state.is_chance_node():
                outcomes = state.chance_outcomes()
                action = np.random.choice([o[0] for o in outcomes], 
                                         p=[o[1] for o in outcomes])
                state.apply_action(action)
            
            # HANDLE SIMULTANEOUS MOVES
            elif state.is_simultaneous_node():
                joint_action = []
                for p in range(cfr_solver.num_players):
                    info_state = state.information_state_string(p)
                    legal_actions = state.legal_actions(p)
                    
                    # Use average strategy
                    if info_state in cfr_solver.info_set_map:
                        strategy = cfr_solver.get_average_strategy(info_state)
                        action_idx = np.random.choice(len(legal_actions), p=strategy)
                        action = legal_actions[action_idx]
                    else:
                        action = np.random.choice(legal_actions)
                    
                    joint_action.append(action)
                
                state.apply_actions(joint_action)
            
            # SEQUENTIAL MOVES
            else:
                current_player = state.current_player()
                info_state = state.information_state_string(current_player)
                legal_actions = state.legal_actions()
                
                # Use average strategy
                if info_state in cfr_solver.info_set_map:
                    strategy = cfr_solver.get_average_strategy(info_state)
                    action_idx = np.random.choice(len(legal_actions), p=strategy)
                    action = legal_actions[action_idx]
                else:
                    action = np.random.choice(legal_actions)
                
                state.apply_action(action)
        
        returns = state.returns()
        total_utility += returns
    
    avg_utility = total_utility / num_samples
    exploitability = np.max(np.abs(avg_utility))
    
    print(f"Average utility: {avg_utility}")
    print(f"Exploitability estimate: {exploitability:.6f}")
    
    if exploitability < 0.01:
        print("✓ Strategy appears to be near Nash equilibrium!")
    elif exploitability < 0.1:
        print("~ Strategy is decent but could improve with more iterations")
    else:
        print("✗ Strategy is highly exploitable, needs more training")
    
    return exploitability



# ============================================
# MAIN EXECUTION
# ============================================

if __name__ == "__main__":
    print("="*60)
    print("COUNTERFACTUAL REGRET MINIMIZATION (CFR)")
    print("Testing on Kuhn Poker (2-player zero-sum)")
    print("="*60)
    print()
    
    # Load Kuhn Poker
    game = pyspiel.load_game("kuhn_poker")
    
    # Create CFR solver
    cfr = CFRSolver(game)
    
    # Train CFR
    NUM_ITERATIONS = 10000
    cfr.train(NUM_ITERATIONS)
    
    # Print learned strategy
    cfr.print_strategy()
    
    # Compute exploitability
    exploitability = compute_exploitability(cfr, num_samples=10000)
    
    print("\n" + "="*60)
    print("VALIDATION COMPLETE")
    print("="*60)
    print(f"\nIf exploitability is low (<0.01), CFR has converged to Nash!")
    print(f"Next step: Test on 3-player games to see if it still works.\n")

COUNTERFACTUAL REGRET MINIMIZATION (CFR)
Testing on Kuhn Poker (2-player zero-sum)

=== CFR Solver Initialized ===
Game: Kuhn Poker
Players: 2
Max actions: 2

=== Starting CFR Training for 10000 iterations ===

Iteration 1000/10000 - Avg game value: 0.000000
Iteration 2000/10000 - Avg game value: 1.500000
Iteration 3000/10000 - Avg game value: 2.000000
Iteration 4000/10000 - Avg game value: 1.500000
Iteration 5000/10000 - Avg game value: 0.000000
Iteration 6000/10000 - Avg game value: 1.000000
Iteration 7000/10000 - Avg game value: 1.500000
Iteration 8000/10000 - Avg game value: -1.500000
Iteration 9000/10000 - Avg game value: 0.000000
Iteration 10000/10000 - Avg game value: 0.000000

=== Training Complete ===
Final average game values: [ 1. -1.]


=== Average Strategy (Nash Equilibrium Approximation) ===

Info Set: 0
  Pass/Check: 0.9984 (99.84%)
  Bet/Call: 0.0016 (0.16%)

Info Set: 0b
  Pass/Check: 0.9997 (99.97%)
  Bet/Call: 0.0003 (0.03%)

Info Set: 0p
  Pass/Check: 0.9997 (99.97%